### Binary classification with Non-Deep learning algorithms: Naive Bayes and SVM
We will use Naive Bayes and Support Vector machine.


In [1]:
!pip install --upgrade numpy
!pip install --upgrade pandas
!pip install pyspark==2.4.5
!pip install -U scikit-learn

Requirement already up-to-date: numpy in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.19.2)
Requirement already up-to-date: pandas in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.1.2)
     |████████████████████████████████| 217.8MB 171kB/s  eta 0:00:01    |████▌                           | 30.3MB 38.4MB/s eta 0:00:05     |████████████▎                   | 83.4MB 30.8MB/s eta 0:00:05     |█████████████████████▉          | 148.3MB 45.8MB/s eta 0:00:02
     |████████████████████████████████| 204kB 34.9MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
Requirement already up-to-date: scikit-learn in /opt/conda/envs/Python36/lib/python3.6/site-packages (0.23.2)


In [2]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [3]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()


In [4]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from pprint import pprint
from time import time
import logging
import numpy as np
import pandas as pd
import string
import re
from datetime import datetime
from packaging import version

from ibm_botocore.client import Config
import ibm_boto3

from sklearn.model_selection import train_test_split

In [3]:
#Get our data from IBM Cloud

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_news = {
    'IAM_SERVICE_ID': 'iam-ServiceId-32e8ee67-397c-4ff1-b69b-543172331f43',
    'IBM_API_KEY_ID': 'Rx4FR4JSAueCnnIsoevsgYgOsuh8LCXtbkFpFpC0EmVU',
    #'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'ENDPOINT':'https://s3-api.us-geo.objectstorage.softlayer.net',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'advanceddatasciencecapstone-donotdelete-pr-tqabpnbxebk8rm',
    'FILE': 'dfTrueFalseNews.pkl'
}

def download_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')

def upload_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.upload_file(Filename=local_file_name, Bucket=credentials['BUCKET'],Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print(' File Uploaded')
        
dfNews = download_file_cos(credentials_news, "dfTrueFalseNews.pkl", "dfTrueFalseNews.pkl")
dfTrueFalseNews_tokenized  = download_file_cos(credentials_news,'dfTrueFalseNews_tokenized.pkl','dfTrueFalseNews_tokenized.pkl')

File Downloaded
File Downloaded


In [5]:
dfNewsTemp = pd.read_pickle('dfTrueFalseNews.pkl')
#dfNews['truthvalue'] = pd.Categorical(dfNews['truthvalue'])
x = dfNewsTemp['text'].values

y = dfNewsTemp['truthvalue'].values

print (dfNewsTemp.shape, dfNewsTemp.columns, '\n', dfNewsTemp.dtypes, type(x), type(y))


(1126, 3) Index(['text', 'source', 'truthvalue'], dtype='object') 
 text          object
source        object
truthvalue    object
dtype: object <class 'numpy.ndarray'> <class 'numpy.ndarray'>


### Prepare the Text
1. Change all the text to lower case
2. Word Tokenization
3. Remove Stop words
4. Remove Non-alpha text
5. Word Lemmatization

In [6]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
from nltk.corpus import wordnet as wn
from collections import defaultdict
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

# reproduce the same result every time the script is run.
np.random.seed(500)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AdamDavis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\AdamDavis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\AdamDavis\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AdamDavis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# Step 1: Change all the text to lower case.
dfNewsTemp['text'] = [entry.lower() for entry in dfNewsTemp['text']]

In [8]:
# Step 2: Tokenization : Each entry will be broken into set of words
dfNewsTemp['text']= [word_tokenize(entry) for entry in dfNewsTemp['text']]

In [9]:
# Step 3,4,5: Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

In [63]:
# Initializing WordNetLemmatizer()
word_Lemmatized = WordNetLemmatizer()
#dfNewsTemp.loc['text_final'].astype('object')
col = []
for index,entry in enumerate(dfNewsTemp['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    #word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        #if word not in stopwords.words('english') and word.isalpha():
        if word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    
    #dfNewsTemp.loc[index,'text_final'] = " ".join(Final_words)
    #dfNewsTemp.loc[index,'text_final'] = Final_words
    col.append(" ".join(Final_words))
dfNewsTemp['text_final'] = col


In [23]:
#dfNewsTemp['text_final'] = col
#col[-1]

In [41]:
# This took a while, so let's save the result.
#dfNewsTemp.to_pickle('dfTrueFalseNews_tokenized_nostopwords.pkl', protocol=4 )
#dfNewsTemp.to_pickle('dfTrueFalseNews_tokenized.pkl', protocol=4 )

# save to our cloud storage 
#upload_file_cos(credentials_news,'dfTrueFalseNews_tokenized_nostopwords.pkl','dfTrueFalseNews_tokenized_nostopwords.pkl')

In [52]:
dfTrueFalseNews_tokenized  = download_file_cos(credentials_news,'dfTrueFalseNews_tokenized.pkl', 'dfTrueFalseNews_tokenized.pkl')
!ls -al

NameError: name 'download_file_cos' is not defined

In [10]:
#dfTrueFalseNews_tokenized = pd.read_pickle('dfTrueFalseNews_tokenized.pkl')
dfTrueFalseNews_tokenized = pd.read_pickle('dfTrueFalseNews_tokenized_nostopwords.pkl')

dfTrueFalseNews_tokenized.head()

,text,source,truthvalue,text_final
tech003legit,"[a, google, computer, victorious, over, the, w...",MihalceaNewsLegit,1,google computer victorious world champion satu...
polit11legit,"[white, house, keeps, up, sanctuary, cities, p...",MihalceaNewsLegit,1,white house keep sanctuary city pressure fundi...
biz40legit,"[why, silicon, valley, is, n't, fighting, to, ...",MihalceaNewsLegit,1,silicon valley fight save internet yet think n...
edu10legit,"[protesters, disrupt, devos, school, visit, pr...",MihalceaNewsLegit,1,protester disrupt devos school visit protester...
tech038legit,"[solar-powered, 'skin, ', could, make, prosthe...",MihalceaNewsLegit,1,could make prosthetics real many people try st...


In [11]:
from sklearn import model_selection
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(dfTrueFalseNews_tokenized['text_final'],dfTrueFalseNews_tokenized['truthvalue'],test_size=0.1, stratify=dfTrueFalseNews_tokenized['truthvalue'], random_state=42, shuffle=True)
#print(dfTrueFalseNews_tokenized.shape, (dfTrueFalseNews_tokenized['text_final'].shape), (dfTrueFalseNews_tokenized['truthvalue'].shape))
#dfTrueFalseNews_tokenized['text_final'][0]

In [56]:
#corpus = [" ".join(i) for i in dfTrueFalseNews_tokenized['text_final']]
#corpus[1:3]
dfTrueFalseNews_tokenized.head()

#dfTrueFalseNews_tokenized.dtypes
#import sklearn
#sklearn.show_versions()

,text,source,truthvalue,text_final
tech003legit,"[a, google, computer, victorious, over, the, w...",MihalceaNewsLegit,1,google computer victorious world champion satu...
polit11legit,"[white, house, keeps, up, sanctuary, cities, p...",MihalceaNewsLegit,1,white house keep sanctuary city pressure fundi...
biz40legit,"[why, silicon, valley, is, n't, fighting, to, ...",MihalceaNewsLegit,1,silicon valley fight save internet yet think n...
edu10legit,"[protesters, disrupt, devos, school, visit, pr...",MihalceaNewsLegit,1,protester disrupt devos school visit protester...
tech038legit,"[solar-powered, 'skin, ', could, make, prosthe...",MihalceaNewsLegit,1,could make prosthetics real many people try st...


In [14]:
# some open source utilities to evaluate a model.
# you can see them at: https://github.com/dipanjanS/practical-machine-learning-with-python/blob/master/notebooks/Ch05_Building_Tuning_and_Deploying_Models/model_evaluation_utils.py
import importlib
#importlib.reload(model_evaluation_utils)
import model_evaluation_utils as meu

### Use Naive Bayes Classifier with ngrams

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
ctVectorizer = CountVectorizer(ngram_range=(1,2), min_df=1 )

ctVectorizer.fit_transform(dfTrueFalseNews_tokenized['text_final'])

Train_X_2ngrams = ctVectorizer.transform(Train_X)
Test_X_2ngrams = ctVectorizer.transform(Test_X)

In [15]:
from sklearn import naive_bayes
from sklearn.metrics import accuracy_score
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Train_y_int = Train_Y.astype('int')
Test_y_int = Test_Y.astype('int')
Naive.fit(Train_X_2ngrams,Train_y_int)# predict the labels on validation dataset
#Naive.fit(Train_X_Tfidf,Train_Y)# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_2ngrams)# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_y_int)* 100)
print("Naive model params: {}".format(Naive.get_params()))
meu.display_model_performance_metrics(true_labels=Test_Y.tolist(), predicted_labels=predictions_NB.tolist())

Naive Bayes Accuracy Score ->  46.017699115044245
Naive model params: {'alpha': 1.0, 'class_prior': None, 'fit_prior': True}
Model Performance metrics:
------------------------------
Accuracy: 0.4602
Precision: 0.4594
Recall: 0.4602
F1 Score: 0.4589

Model Classification report:
------------------------------
              precision    recall  f1-score   support

           1       0.45      0.41      0.43        56
           0       0.47      0.51      0.49        57

    accuracy                           0.46       113
   macro avg       0.46      0.46      0.46       113
weighted avg       0.46      0.46      0.46       113


Prediction Confusion Matrix:
------------------------------
          Predicted:    
                   1   0
Actual: 1         23  33
        0         28  29


### Naive Bayes Classifier with tf-idf vectorizer

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
Tfidf_vect = TfidfVectorizer()

Tfidf_vect.fit(dfTrueFalseNews_tokenized['text_final'])

Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [71]:
from sklearn import naive_bayes
from sklearn.metrics import accuracy_score
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Train_y_int = Train_Y.astype('int')
Test_y_int = Test_Y.astype('int')
Naive.fit(Train_X_Tfidf,Train_y_int)# predict the labels on validation dataset
#Naive.fit(Train_X_Tfidf,Train_Y)# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_y_int)* 100)
print("Naive model params: {}".format(Naive.get_params()))
meu.display_model_performance_metrics(true_labels=Test_Y.tolist(), predicted_labels=predictions_NB.tolist())

Naive Bayes Accuracy Score ->  46.017699115044245
Naive model params: {'alpha': 1.0, 'class_prior': None, 'fit_prior': True}
Model Performance metrics:
------------------------------
Accuracy: 0.4602
Precision: 0.4584
Recall: 0.4602
F1 Score: 0.4563

Model Classification report:
------------------------------
              precision    recall  f1-score   support

           1       0.45      0.38      0.41        56
           0       0.47      0.54      0.50        57

    accuracy                           0.46       113
   macro avg       0.46      0.46      0.46       113
weighted avg       0.46      0.46      0.46       113


Prediction Confusion Matrix:
------------------------------
          Predicted:    
                   1   0
Actual: 1         21  35
        0         26  31


### Discussion
This is a Naive Bayes model using a Bag of Words TF-IDF encoding. 

The result of 46% accuracy is not very good.  

## Use Support Vector Machine
We use a tf-idf embedding.

In [19]:
from sklearn import svm
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=2.5, kernel='sigmoid', degree=3, gamma='scale')
SVM.fit(Train_X_Tfidf,Train_y_int)# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> {}".format(accuracy_score(predictions_SVM, Test_y_int)*100))
print("SVM model params: {}".format(Naive.get_params()))
meu.display_model_performance_metrics(true_labels=Test_Y.tolist(), predicted_labels=predictions_SVM.tolist())

SVM Accuracy Score -> 59.29203539823009
SVM model params: {'alpha': 1.0, 'class_prior': None, 'fit_prior': True}
Model Performance metrics:
------------------------------
Accuracy: 0.5929
Precision: 0.6035
Recall: 0.5929
F1 Score: 0.5889

Model Classification report:
------------------------------
              precision    recall  f1-score   support

           1       0.64      0.49      0.56        59
           0       0.56      0.70      0.62        54

    accuracy                           0.59       113
   macro avg       0.60      0.60      0.59       113
weighted avg       0.60      0.59      0.59       113


Prediction Confusion Matrix:
------------------------------
          Predicted:    
                   1   0
Actual: 1         29  30
        0         16  38


### Discussion

This is a Support Vector Machine using the same Bag of Words TF-IDF encoding as above and does surprisingly better, with a 59% accuracy rate that rivals others results.



### Split the data

In [ ]:
# use the dataframe to write a parquet file
dfNewsTemp.to_parquet('dfTrueFalseNews_tokenized.parquet')
dfNews = spark.read.parquet('dfTrueFalseNews_tokenized.parquet')
dfNews.createOrReplaceTempView('dfNews')

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(dfNewsTemp['text_final'],Corpus['label'],test_size=0.1)

In [ ]:
splits = dfNews.randomSplit([0.9, 0.1])
df_train = splits[0]
df_test = splits[1]

